Jupyter Notebok tweaks for the presentation export with Reveal.js and when using RISE (RISE is a jupyter notebook extension).

**Important**: this must be as slide type "Notes"

In [2]:
from IPython.display import HTML

def apply_html_tweak(html_script_path):
    with open(html_script_path, encoding='utf-8') as fh:
        display(HTML(fh.read()))

#apply_html_tweak("../../reveal_js_tweaks.txt")
apply_html_tweak("../../rise_tweaks.txt")

<span style="font-size:4em;color:#130754;">Pandas crash course</span>


<img src="./imgs/pandas_crash_course_logo.jpg" alt="Pandas crash course logo" width="500">


<i>By <b>Thibault Bétrémieux</b>, Data Scientist at port-neo Freiburg GmbH (part of port-neo GmbH), thibault.betremieux@port-neo.com</i>

This is a presentation for the Python User Group Freiburg, see [Meetup Link](https://www.meetup.com/fr-FR/Python-User-Group-Freiburg/events/275586196/).

**[Link to the resources](https://github.com/ThibTrip/thib/tree/master/2021/pandas_crash_course)**

# Table of contents

## Part 1 of the Meetup

1. Introduction: explanations on the main structures of the pandas API (Series, DataFrame, indices, data types), resources that can help you

2. I/O: read from/save to csv, Excel, SQL, ... BONUS: SQL records update using DataFrames

3. Selecting rows/columns

4. Basic math operations

5. Some common DataFrame operations e.g. handling duplicates, missing values

## Part 2 of the Meetup (T.B.A.)

6\. String operations

7\. Time operations

8\. Transformations: concatenate, groupby, pivot, pivot_table, stack

9\. Using custom functions: map, apply, applymap

10\. Outside of scope: things you should know about that I won't present in detail
    - pandas' method chaining super power
    - categoricals
    - plotting
    - styling

# 1. Introduction

## 1.1. What is pandas and what can you do with it

> pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series.
> 
> (Source: [Wikipedia](https://en.wikipedia.org/wiki/Pandas_%28software%29))


pandas was created by Wes McKinney.

Amongst many other things pandas allows you to:

* **read from and save to** many formats (csv, excel, SQL, feather, parquet, ...)
* do fast vectorized operations (e.g. column1 + column2) with lots of already integrated math/stats methods
* transform data by grouping (similar to SQL GROUP BY), pivoting, stacking (columns to rows) and unstacking, ...
* apply your own functions element wise or vector wise
* work with strings and datetimes easely (e.g. `pd.Series.str.strip`, `pd.Series.dt.month`, ...)
* deal with duplicates and null values
* plot
* present tables with pretty styling 🦄

If used correctly (i.e. in a vectorized and optimized manner) it can be very fast. It actually uses `C` extensions which speeds up operations.

## 1.2. What you can't do (or do well) with pandas

### 1.2.1. Working with massive datasets 
You can't transform very big tabular data at once (e.g. a dataset of 100GB) since you'd have to fit it in memory. But there are easy ways to overcome such situations

* work with pandas in chunks (read chunk, process chunk, save chunk)
* pandas + something else. E.g. dump the data in a SQL database and load an aggregated version of it in pandas where you will do additional transformations.
* something else e.g. [ibis](http://ibis-project.org/) which combines Python syntax with SQL. Or simply SQL (I myself use PostgreSQL a lot 🐘)
* using libraries such as [dask](https://docs.dask.org/en/latest/dataframe.html). `dask` is **lazy** and only does the computations on the dataset when needed
* see also [page "Scaling to large datasets" in pandas docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html)


### 1.2.2. Structures with many dimensions

Working with structures with more dimensions that just a plain table with rows and columns will be tough with pandas. E.g. something like an OLAP cube: 

![olap](./imgs/olap_cube.png)

The example above could still work in pandas because you can use multiple levels of columns and indices but it would start to get tricky 🤔. For such structures you can use [xarray](http://xarray.pydata.org/).

## 1.3. Installation

`pip install pandas` or `conda install pandas` (if you use `conda`)

This tutorial uses the most recent version of pandas which is **`1.2.1`** at this time.

## 1.4. Recommandations

When starting/updating a script/module/library I recommand using `pandas` in an interactive environment such as [Jupyter Lab](https://jupyter.org/) where you can **watch and document the transformation of a table** which makes it very easy to troubleshoot problems.
The easiest way to install `Jupyter Lab` is to enter `pip install jupyterlab` but I recommand installing [Anaconda](https://www.anaconda.com/products/individual) which is a bundle containing Python, Jupyter, the conda package and environment manager and lots of other very useful tools for data science.


With my favorite extension for `Jupyter Lab` called [jupytext](https://github.com/mwouts/jupytext) you can even **work with Python text files interactively, as if they were notebooks**. This allows you to develop modules or even libraries which I do myself very often. I talked about it during my last [presentation](https://thibtrip.github.io/lightning_talks/#/5).

## 1.5. Highly recommanded resources

* [The new pandas documentation](https://pandas.pydata.org/docs/getting_started/index.html): I've linked here the **"Getting started"** page which is well illustrated but the whole documentation is really awesome (also the reference for functions/methods etc, despite some abstract examples sometimes)


**WARNING:** in the resources below some things may be outdated and some things can now be done more efficiently or with a shorter code but those are still great resources nonetheless.

* data school ([website](https://www.dataschool.io/easier-data-analysis-with-pandas/) | [YouTube channel](https://www.youtube.com/c/dataschool/featured)): topics on pandas that are very well explained and presented in video
* [Python for Data Analysis Book](https://wesmckinney.com/pages/book.html) **by the author of pandas**: make sure to get the latest edition (2nd at the time I am writing this)! This is not as easy as the data school videos but it's very helpful

Since `pandas` is a very popular library, **most questions** (especially basic ones) are just **one google query away**. And most of the time you will propbably end up on a **stackoverflow** thread 🙈 or the **pandas documentation**.

## 1.6. Structures

### 1.6.1. `pd.Series`

> Series is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the index.
>
> (Source: [pandas docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html))

This can be compared to a dictionary. Unlike the dictionary the index allows duplicate by default! A [new experimental feature](https://pandas.pydata.org/docs/whatsnew/v1.2.0.html#optionally-disallow-duplicate-labels) exists to check that automatically on subsequent operations.

#### Creating a Series from scratch

In [3]:
import pandas as pd

s = pd.Series(['Thibault', 'John'])
s

0    Thibault
1        John
dtype: object

In [4]:
s.index

RangeIndex(start=0, stop=2, step=1)

In [5]:
s.values

array(['Thibault', 'John'], dtype=object)

you can provide a name for the Series, a different index than the default one (range from 0 to the length of the DataFrame minus one) and even name the index

In [6]:
s = pd.Series(data=['Thibault', 'John'], index=['a', 'b'], name='firstname')
s.index.name = 'id'
s

id
a    Thibault
b        John
Name: firstname, dtype: object

In [7]:
pd.Series(data={'a':'Thibault', 'b':'John'}, # a dict {index:value} also works
          name='firstname').rename_axis(index='id')

id
a    Thibault
b        John
Name: firstname, dtype: object

In [8]:
s.dtype # 'O' stands for "object", more on that later

dtype('O')

#### Notes on data types

wondering why the data type of our Series is "O" (**"object"**) and not something like **"string"** or "str"? `pandas` uses lots of `numpy` arrays in the backend and `numpy` does not have a string data type for arrays.

By the way `numpy` arrays are similar to python lists but they are typed which allows for much faster vectorized operations (more on that later). And `numpy` uses `C` extensions which is faster than Python.

In [9]:
type(s.values) # yep it's a numpy array

numpy.ndarray

But pandas has developped so called extension arrays (they extend numpy arrays) where a **"string"** data type exists. You can start using them already by using the method `convert_dtypes`

Why it matters:

* the numpy "object" dtype (data type) is not specific to strings so it can contain other things
* pandas' string data type will only allow string or null
* it's clear what it is (string and not something else)

In [10]:
s = s.convert_dtypes()
s

id
a    Thibault
b        John
Name: firstname, dtype: string

In [11]:
s.dtype # StringDtype is an extension array of pandas

StringDtype

### 1.6.2. `pd.DataFrame`

> Data structure also contains labeled axes (rows and columns) [...] Can be thought of as a dict-like container for Series objects. The primary pandas data structure.
>
> (Source: [pd.DataFrame docstring](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html))

#### Creating a DataFrame from scratch

I'll use different ways to create DataFrames later. Pandas can ingest many different things 🐼.

Like `pd.Series` you can put pretty much any type of data you want in a DataFrame (float, int, complex numbers, tuple, list, str, ...).

In [12]:
data = {'first_name':['Alex', 'John'],
        'number_of_cars':[0, 2],
        'is_subscribed_to_newsletter':[False, True],
        'interests':[['pizza', 'traveling'], ['movies']]}
df = pd.DataFrame(data=data)
df

,first_name,number_of_cars,is_subscribed_to_newsletter,interests
0,Alex,0,False,"[pizza, traveling]"
1,John,2,True,[movies]


In [13]:
df.index

RangeIndex(start=0, stop=2, step=1)

In [14]:
df.columns

Index(['first_name', 'number_of_cars', 'is_subscribed_to_newsletter',
       'interests'],
      dtype='object')

## Data types

In [15]:
df.dtypes

first_name                     object
number_of_cars                  int64
is_subscribed_to_newsletter      bool
interests                      object
dtype: object

In [16]:
# I'd suggest using the new data types here as well
df = df.convert_dtypes()
df.dtypes

first_name                      string
number_of_cars                   Int64
is_subscribed_to_newsletter    boolean
interests                       object
dtype: object

#### The columns of the DataFrame are in fact Series 😳

In [17]:
df['first_name'] # selects the column "first_name"

0    Alex
1    John
Name: first_name, dtype: string

In [18]:
type(df['first_name'])

pandas.core.series.Series

### 1.6.3. Notes on indices and labels

* The index of a Series or a DataFrame can have multiple levels
* The columns of a DataFrame can have multiple levels
* A DataFrame can also receive a name for each column level and a name for the index. Yes I know it's confusing 🐒

Some operations may return a Series or DataFrame with multidimensional headers or indices (e.g. pivot, stack) so do not be surprised!
Here is an example which we won't work with because we'd need much more time if we wanted to learn about selecting and operating on DataFrames with multidimensional headers/indices

In [19]:
import numpy as np
index = pd.MultiIndex.from_product([[2020, 2021], ['Freiburg', 'Lindau']], names=['year', 'location'])
columns = pd.MultiIndex.from_product([['Superstore', 'Alnatura'], ['cosmetics', 'food']], names=['store', 'product_type'])
data = np.random.randint(low=10000, high=900000, size=(len(index),len(columns)))
df = pd.DataFrame(data=data, index=index, columns=columns)
# add a label for the whole index (not labels for each level which we have already)
df.index.name = 'my_index'
df

store         Superstore          Alnatura        
product_type   cosmetics    food cosmetics    food
year location                                     
2020 Freiburg     810559  486073    310653  319119
     Lindau       395859  700673    253274   22629
2021 Freiburg     840583   89122    563113  432098
     Lindau       727061  538181    223067  782422

In [20]:
# it's surprising pandas doesn't show the index name but only the names of the levels, but as you can see it's here
print(f'index name: {df.index.name}')
print(f'names of index levels: {df.index.names}')

index name: my_index
names of index levels: ['year', 'location']


The behavior of some operations changes with multidimensional headers and indices.

For instance when using the syntax I mentioned before for selecting a column we get a DataFrame back in the example below (a subset of our DataFrame to be precise) and not a Series

In [21]:
df['Superstore']

product_type   cosmetics    food
year location                   
2020 Freiburg     810559  486073
     Lindau       395859  700673
2021 Freiburg     840583   89122
     Lindau       727061  538181

See also this [example](https://github.com/ThibTrip/thib/blob/master/2021/pandas_crash_course/examples/merge/question_pandas_merge.ipynb) on a question about merge operation Lilia Schuster asked me!

The transformation is a bit complicated but the **multidimensional DataFrame** output at cell **`[5]`** is particularly interesting.

### 1.6.4. ⚠️ The index is a separate entity and cannot be selected like columns!

In [22]:
import traceback

df = pd.DataFrame({'name':['Tibaldo', 'John']}, index=pd.Index([100, 200], name='id'))
display(df) # pretty print function implicitely imported when using Jupyter/IPython (from IPython.display import display)

try:
    df['id']
except KeyError:
    traceback.print_exc()

df.index # <-- select like this (and if you have multiple levels use df.index.get_level_values('id'))

,name
id,
100,Tibaldo
200,John


Traceback (most recent call last):
  File "/home/tibaldo/anaconda3/envs/lab-env/lib/python3.8/site-packages/pandas/core/indexes/base.py", line 2891, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'id'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-22-1789fb35f70c>", line 7, in <module>
    df['id']
  File "/home/tibaldo/anaconda3/envs/lab-env/lib/python3.8/site-packages/pandas/core/frame.py", line 2902, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/home/tibaldo/anaconda3/envs/lab-

Int64Index([100, 200], dtype='int64', name='id')

# 2. I/O (Input/Output)

**WARNING**: with csv and excel files (and any kind of file that does not contain data type indications actually) pandas infers data types. Values such as `01234` will be interpreted as `1234`. If it was a zip code it is now wrong.

Use the parameter **`dtype`** to circumvent this as I'll show in the examples or use the [gist](https://gist.github.com/ThibTrip/55fce86de023c98b9379de2f9be58249) I made for that which will infer which columns should be kept as string by using some custom rules.

In [23]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///./data/db.sqlite3')

# prepare some data to read
# note: for once I don't use UTC because we cannot include timezone info in Excel...
data = [['Alex', pd.Timestamp('2020-01-01 10:01:02'), 0, False, '01234', '+999'],
        ['John', pd.Timestamp('2020-01-01 20:07:10'), 2, True, '12345', '+222']]
columns = ['first_name', 'last_website_visit', 'number_of_cars', 'is_subscribed_to_newsletter',
           'zip_code', 'phone_number']
df = pd.DataFrame(data=data, columns=columns, index=pd.Index((100, 200), name='id'))

df.to_csv('./data/example.csv', sep=';') # ";" is common in Germany so people ought to now about it
df.to_excel('./data/example.xlsx', sheet_name='example')
df.to_pickle('./data/df.pickle')
df.to_sql(name='people', con=engine, if_exists='replace', method='multi')

## 2.1. CSV files


Explanations on some parameters you'll probably use at some point:

* `sep`: delimiter, by default `","` but in Germany it is often `";"`
* `encoding`: in Germany it is sometimes "iso-8859-1" (encoding can be detected with [chardet](https://github.com/chardet/chardet) if needed)
* **`dtype`**: which data type pandas should use for columns e.g. `string` (all the columns as the String data type) or `{'zip_code':'string'}` (the column "zip_code" as the String data type, for the rest we let pandas infer it)
* `index_col`: which column(s) to use as index
* `usecols` for selecting only some columns
* `chunksize`: yields n rows at a time (gives back an iterator), this is great for working on huge files in chunks
* `skiprows`: list-like of line numbers or number of lines to skip (or callable)

Obviously there are many more parameters e.g. I remember tweaking `na_values` and `keep_default_na` for working with geographic data to avoid `NA` (Namibia) being interpreted as a NULL value 🙈.

In [24]:
# read from csv
df = pd.read_csv('./data/example.csv', sep=';', dtype={'zip_code':'string', 'phone_number':'string'}, index_col='id')

# save to csv (let's assume you'll want ';' as a separator as well here)
df.to_csv('./data/example.csv', sep=';')

df

,first_name,last_website_visit,number_of_cars,is_subscribed_to_newsletter,zip_code,phone_number
id,,,,,,
100,Alex,2020-01-01 10:01:02,0,False,01234,+999
200,John,2020-01-01 20:07:10,2,True,12345,+222


you can read directly from a URL too (ftp, s3 and more URL schemes are also supported). Most read and save methods also handle **compression**. Note that reading from archives only works if there is only one file in the archive (but it's easy to [use a file handle to select files](https://stackoverflow.com/a/44118138))

In [25]:
df = pd.read_csv('http://download.geonames.org/export/dump/cities15000.zip',
                 header=None, # the first row is not a header, I'll provide my own names (see below)
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class',
                        'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code',
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                 sep='\t') # tabulation

df = df.convert_dtypes().set_index('geonameid')
df

,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
geonameid,,,,,,,,,,,,,,,,,,
3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,<NA>,08,<NA>,<NA>,<NA>,15853,<NA>,1033,Europe/Andorra,2008-10-15
3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,<NA>,07,<NA>,<NA>,<NA>,20430,<NA>,1037,Europe/Andorra,2020-03-03
290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,<NA>,07,<NA>,<NA>,<NA>,62747,<NA>,2,Asia/Dubai,2019-10-24
291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,<NA>,05,<NA>,<NA>,<NA>,351943,<NA>,2,Asia/Dubai,2019-09-09
291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,<NA>,01,103,<NA>,<NA>,63482,<NA>,124,Asia/Dubai,2019-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894701,Bulawayo,Bulawayo,"BUQ,Bulavajas,Bulavajo,Bulavejo,Bulawayo,bu la...",-20.15000,28.58333,P,PPLA,ZW,<NA>,09,<NA>,<NA>,<NA>,699385,<NA>,1348,Africa/Harare,2019-09-05
895061,Bindura,Bindura,"Bindura,Bindura Town,Kimberley Reefs,Биндура",-17.30192,31.33056,P,PPLA,ZW,<NA>,03,<NA>,<NA>,<NA>,37423,<NA>,1118,Africa/Harare,2010-08-03
895269,Beitbridge,Beitbridge,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",-22.21667,30.00000,P,PPL,ZW,<NA>,07,<NA>,<NA>,<NA>,26459,<NA>,461,Africa/Harare,2013-03-12


## 2.2. Excel files

Apart from `sep` and `encoding` this method also has the parameters I mentioned above.

A very important argument here is `sheet_name` to select which sheet of the excel file you want to read.

You'll have to install `openpyxl` for reading and writing Excel files (`pip install openpyxl`). Openpyxl is pretty cool. I've used it a few times to list sheet names, check out which sheet are hidden, how cells are formatted etc. Here is for instance an [Openpyxl tutorial](https://zetcode.com/python/openpyxl/) showing some of the possibilities.

If working with `.xls` files (as opposed to `.xlsx`) you'll have to use `xlrd` (`pip install xlrd`).

In [26]:
df = pd.read_excel('./data/example.xlsx', dtype={'zip_code':'string', 'phone_number':'string'}, index_col='id',
                   sheet_name='example') # if not provided, the first sheet is read

# save to excel
df.to_excel('./data/example.xlsx', sheet_name='example') # default sheet name is "Sheet1"

df

,first_name,last_website_visit,number_of_cars,is_subscribed_to_newsletter,zip_code,phone_number
id,,,,,,
100,Alex,2020-01-01 10:01:02,0,False,01234,+999
200,John,2020-01-01 20:07:10,2,True,12345,+222


## 2.3. SQL

We use the library `sqlalchemy` for creating an object to communicate with the database (see `engine` below).

The `sqlalchemy.create_engine` function we use below takes a [connection string to a database](https://docs.sqlalchemy.org/en/latest/core/engines.html).

For example, the syntax for PostgreSQL (it's similar for most other databases) is as follows:

```python
connection_string = f'postgresql://{USERNAME}:{PASWORD}@{HOST}:{PORT}/{DB}'
```

In [27]:
from sqlalchemy import create_engine # pip install sqlalchemy

# sqlite is a bit of special SQL flavor since the database is on the disk (so you just provide a path)
connection_string = 'sqlite:///./data/db.sqlite3'
engine = create_engine(connection_string)

df = pd.read_sql(sql='SELECT * FROM people;', con=engine, index_col='id')
df

,first_name,last_website_visit,number_of_cars,is_subscribed_to_newsletter,zip_code,phone_number
id,,,,,,
100,Alex,2020-01-01 10:01:02.000000,0,0,01234,+999
200,John,2020-01-01 20:07:10.000000,2,1,12345,+222


Saving to SQL

In [28]:
df.to_sql(name='people',
          con=engine,
          index=True, # default, we want to save the index 
          if_exists='replace', # what to do if the table exists (default is "fail" which raises an Exception) and there is also "append"
          method='multi') # IMPORTANT!!!! Otherwise inserts row by row which is super slow

**Important tip**: I made a [benchmark of df.to_sql](https://gist.github.com/ThibTrip/b0a752c7b7b1756d87832a5bb48a165b) for PostgreSQL with many methods and as you'll see there are huge differences: from 2min 48 for the slowest method down to **16.1s** for the fastest method!

### Bonus: updating rows in a SQL table with a DataFrame thanks to my library [pangres](https://github.com/ThibTrip/pangres)

You will need a **primary key** (or a unique key) which is one or multiple columns for identifying records. The primary key will have to be set as index in the pandas DataFrame.

The example demonstrates what the library can do, but you'll definitely need the [documentation](https://github.com/ThibTrip/pangres/wiki).

load some table I saved in the database

In [29]:
from sqlalchemy import create_engine # pip install sqlalchemy
engine = create_engine('sqlite:///./data/db.sqlite3')
engine.execute('DROP TABLE IF EXISTS pangres_test;')
engine.execute("""CREATE TABLE pangres_test (
                      id int PRIMARY KEY,
                      first_name text,
                      number_of_cars int);""")
engine.execute("""INSERT INTO pangres_test (id, first_name, number_of_cars)
                  VALUES (100, "John", 2), (200, "Thibault", 1);""")

In [30]:
from sqlalchemy import create_engine # pip install sqlalchemy
from pangres import upsert # pip install pangres
engine = create_engine('sqlite:///./data/db.sqlite3') # "./data/db.sqlite3" is the path to a SQlite3 database
df = pd.read_sql('SELECT * FROM pangres_test;', con=engine, index_col='id').convert_dtypes()
df

,first_name,number_of_cars
id,,
100,John,2
200,Thibault,1


create some new data to update and insert

In [31]:
new_df = pd.DataFrame({'id':[200, 300], 'first_name':['Lord Thibault', 'Alex'], 'number_of_cars':[30, 1], 'number_of_houses':[5, 2]}).set_index('id')
new_df

,first_name,number_of_cars,number_of_houses
id,,,
200,Lord Thibault,30,5
300,Alex,1,2


upsert (insert update) the new data and check what is in the table

In [32]:
upsert(engine=engine, df=new_df, table_name='pangres_test', if_row_exists='update', add_new_columns=True)
pd.read_sql('SELECT * FROM pangres_test;', con=engine, index_col='id').convert_dtypes()

2021-02-24 19:11:57,401 | INFO     | pangres    | logger:log:48 - Added column pangres_test.number_of_houses (type: BIGINT) in table pangres_test (schema="None")
2021-02-24 19:11:57,403 | INFO     | pangres    | logger:log:37 - Reduced chunksize from 10000 to 249 due to SQlite max variable restriction (max 999).


,first_name,number_of_cars,number_of_houses
id,,,
100,John,2,<NA>
200,Lord Thibault,30,5
300,Alex,1,2


## 2.4. Pickles

While you can use `joblib` to store a Python session on disk, I find pickles really interesting for simply exchanging a few DataFrames (amongst other things) between notebooks.

For pickling other things than DataFrames see [this stackoverflow post](https://stackoverflow.com/a/11218504).

![pickles](./imgs/pickle.png)

In [33]:
df = pd.read_pickle('./data/df.pickle') # note: this will work on any kind of pickled object if you are lazy 🐒 (so not just DataFrames)
df.to_pickle('./data/df.pickle')
df

,first_name,last_website_visit,number_of_cars,is_subscribed_to_newsletter,zip_code,phone_number
id,,,,,,
100,Alex,2020-01-01 10:01:02,0,False,01234,+999
200,John,2020-01-01 20:07:10,2,True,12345,+222


## 2.5. HTML

`pd.read_html` always returns a list of DataFrames: one DataFrame for each table that can be found on given website or HTML code

Example: grab the german region codes (ISO 3166-2) from Wikipedia

In [34]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-2:DE')
dfs[0].set_index('Code')

,Subdivision Name (de),Subdivision Name (en)[note 1]
Code,,
DE-BW,Baden-Württemberg,Baden-Württemberg
DE-BY,Bayern,Bavaria
DE-BE,Berlin,Berlin
DE-BB,Brandenburg,Brandenburg
DE-HB,Bremen,Bremen
DE-HH,Hamburg,Hamburg
DE-HE,Hessen,Hesse
DE-MV,Mecklenburg-Vorpommern,Mecklenburg-Western Pomerania
DE-NI,Niedersachsen,Lower Saxony


## 2.6. Many more file types are handled by pandas

JSON, stata, parquet, feather, HD5F... See [I/O page](https://pandas.pydata.org/pandas-docs/stable/reference/io.html) of the documentation.

## 2.7. Pro tip: `pd.json_normalize`

The function `pd.json_normalize` allows you to quickly parse data from APIs that use JSON

In [35]:
import requests, json
# get facts on cats from a free public API
response = requests.get('https://cat-fact.herokuapp.com/facts')
json_data = response.text
json_data[0:500] + '...' # let's just display a part of the JSON string

'[{"status":{"verified":true,"sentCount":1,"feedback":""},"type":"cat","deleted":false,"_id":"5887e1d85c873e0011036889","user":"5a9ac18c7478810ea6c06381","text":"Cats make about 100 different sounds. Dogs make only about 10.","__v":0,"source":"user","updatedAt":"2020-09-03T16:39:39.578Z","createdAt":"2018-01-15T21:20:00.003Z","used":true},{"status":{"verified":true,"sentCount":1},"type":"cat","deleted":false,"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats ...'

In [36]:
json_data_loaded = json.loads(json_data)
json_data_loaded[0]

{'status': {'verified': True, 'sentCount': 1, 'feedback': ''},
 'type': 'cat',
 'deleted': False,
 '_id': '5887e1d85c873e0011036889',
 'user': '5a9ac18c7478810ea6c06381',
 'text': 'Cats make about 100 different sounds. Dogs make only about 10.',
 '__v': 0,
 'source': 'user',
 'updatedAt': '2020-09-03T16:39:39.578Z',
 'createdAt': '2018-01-15T21:20:00.003Z',
 'used': True}

Use `pd.json_normalize` on the data we just acquired and loaded as JSON

There are very useful arguments in this function e.g. `record_path` to precise which json key holds the records.

Notice the two coluns `status.verified` and `status.sentCount` which came from a structure like this:

```python
{'status': {'verified': True, 'sentCount': 1}}
```
 
The `.` shows the different nodes (you can modify the symbol with the argument `sep`).

In [37]:
df = pd.json_normalize(json_data_loaded) # 
df.head(2) # display the first 2 rows (by default it is 5)

,type,deleted,_id,user,text,__v,source,updatedAt,createdAt,used,status.verified,status.sentCount,status.feedback
0,cat,False,5887e1d85c873e0011036889,5a9ac18c7478810ea6c06381,Cats make about 100 different sounds. Dogs mak...,0,user,2020-09-03T16:39:39.578Z,2018-01-15T21:20:00.003Z,True,True,1,
1,cat,False,588e746706ac2b00110e59ff,588e6e8806ac2b00110e59c3,Domestic cats spend about 70 percent of the da...,0,user,2020-08-26T20:20:02.359Z,2018-01-14T21:20:02.750Z,True,True,1,NaN


Note: this is not the best example since we can also do it in one line 🙈. Well to be fair the **status** column is not split in 2 unlike previously! But you get the idea...

In [38]:
df = pd.read_json('https://cat-fact.herokuapp.com/facts')
df.head()

,status,type,deleted,_id,user,text,__v,source,updatedAt,createdAt,used
0,"{'verified': True, 'sentCount': 1, 'feedback':...",cat,False,5887e1d85c873e0011036889,5a9ac18c7478810ea6c06381,Cats make about 100 different sounds. Dogs mak...,0,user,2020-09-03T16:39:39.578Z,2018-01-15T21:20:00.003Z,True
1,"{'verified': True, 'sentCount': 1}",cat,False,588e746706ac2b00110e59ff,588e6e8806ac2b00110e59c3,Domestic cats spend about 70 percent of the da...,0,user,2020-08-26T20:20:02.359Z,2018-01-14T21:20:02.750Z,True
2,"{'verified': True, 'sentCount': 1}",cat,False,58923f2fc3878c0011784c79,5887e9f65c873e001103688d,I don't know anything about cats.,0,user,2020-08-23T20:20:01.611Z,2018-02-25T21:20:03.060Z,False
3,"{'verified': True, 'sentCount': 1}",cat,False,5894af975cdc7400113ef7f9,5a9ac18c7478810ea6c06381,The technical term for a cat’s hairball is a b...,0,user,2020-11-25T21:20:03.895Z,2018-02-27T21:20:02.854Z,True
4,"{'verified': True, 'sentCount': 1}",cat,False,58e007cc0aac31001185ecf5,58e007480aac31001185ecef,Cats are the most popular pet in the United St...,0,user,2020-08-23T20:20:01.611Z,2018-03-01T21:20:02.713Z,False


# 3. Selecting rows/columns

## Let's make an example DataFrame

Yes the index is not alphabetically sorted, this is on purpose 🐒

In [39]:
records = [['Alex', 0, False, 1],
           ['John', 2, True, 1],
           ['Thib', 0, True, 2],
           ['Arnold', 2, True, 1]]
columns = ['first_name', 'number_of_cars', 'is_subscribed_to_newsletter', 'number_of_houses']
index = pd.Index(data=(400, 265, 450, 122), name='id')

df = pd.DataFrame(data=records, columns=columns, index=index)
df

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
400,Alex,0,False,1
265,John,2,True,1
450,Thib,0,True,2
122,Arnold,2,True,1


## 3.1. `df.head()` and `df.tail()`

head displays the first n rows of a DataFrame (by default 5)

In [40]:
df.head(2)

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
400,Alex,0,False,1
265,John,2,True,1


tail displays the last n rows of a DataFrame (by default 5)

In [41]:
df.tail(2)

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
450,Thib,0,True,2
122,Arnold,2,True,1


## 3.2. simple column selection

In [42]:
df['first_name'] # I selected one label via a string -> returns a Series


id
400      Alex
265      John
450      Thib
122    Arnold
Name: first_name, dtype: object

In [43]:
df[['first_name', 'number_of_cars']] # mutiple columns -> returns a DataFrame


,first_name,number_of_cars
id,,
400,Alex,0
265,John,2
450,Thib,0
122,Arnold,2


In [44]:
df[['first_name']] # I selected one label but via a list -> also returns a DataFrame


,first_name
id,
400,Alex
265,John
450,Thib
122,Arnold


## 3.3. selection by labels (`df.loc`)

This works by using `df.loc`. The basic syntax is `df.loc[rows (indices), columns]`

In [45]:
indices = [400, 265]
columns = ['first_name', 'number_of_cars']
df.loc[indices, columns]

,first_name,number_of_cars
id,,
400,Alex,0
265,John,2


slices are also allowed (they are somewhat similar to slices with lists)

In [46]:
df.loc[:, ['first_name', 'number_of_cars']] # all rows and the columns "first_name" and "number_of_cars"

,first_name,number_of_cars
id,,
400,Alex,0
265,John,2
450,Thib,0
122,Arnold,2


In [47]:
df.loc[400:450, :] # rows from label 400 to label 450

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
400,Alex,0,False,1
265,John,2,True,1
450,Thib,0,True,2


In [48]:
df.loc[:, 'first_name':'is_subscribed_to_newsletter'] # columns from label "first_name" until label "is_subscribed_to_newsletter"

,first_name,number_of_cars,is_subscribed_to_newsletter
id,,,
400,Alex,0,False
265,John,2,True
450,Thib,0,True
122,Arnold,2,True



⚠️ if you select a single row label or a single column label a Series is returned

In [49]:
df.loc[400, :]

first_name                      Alex
number_of_cars                     0
is_subscribed_to_newsletter    False
number_of_houses                   1
Name: 400, dtype: object

In [50]:
df.loc[[400], :] # see how this returns a DataFrame even with a single row label just because I put a list

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
400,Alex,0,False,1


In [51]:
df.loc[:, 'first_name'] # same as df['first_name']

id
400      Alex
265      John
450      Thib
122    Arnold
Name: first_name, dtype: object

🐍 and if you select a single row label AND a single column label you get the value of the cell back.

Here we are effectively selecting a cell and getting the first name of the person with the id 400 (a string)

In [52]:
df.loc[400, 'first_name']

'Alex'

## 3.4. selection by position (`df.iloc`)

In [53]:
index_positions = [0, 2]
columns_positions = [0, 1]
df.iloc[index_positions, columns_positions]

,first_name,number_of_cars
id,,
400,Alex,0
450,Thib,0


you can also use slices here


⚠️ similarly to the standard `range` function `0:2` means only first and second row (not the third one)!

In [54]:
list(range(0, 3)) # this is in case you forgot how range works 🙈

[0, 1, 2]

In [55]:
df.iloc[0:3, :] # rows 0 to 2 included, all columns

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
400,Alex,0,False,1
265,John,2,True,1
450,Thib,0,True,2


In [56]:
df.iloc[:, 0:3] # all rows, columns 0 to 2 included

,first_name,number_of_cars,is_subscribed_to_newsletter
id,,,
400,Alex,0,False
265,John,2,True
450,Thib,0,True
122,Arnold,2,True


## 3.5. selection by masks (`df.loc`)


⚠️ Use `&` and not `and`!

⚠️ Use `|` and not `or`!

⚠️ Parentheses are important! We are doing comparisons array wise so make sure it is "understandable by Python". There is a WikiPedia article I saw a long time about this logic but I couldn't find it again sorry.

**Bad example**

```python
df['money'] > 10000000 or df['first_name'] == 'Thibault'
```

**Good example**

```python
(df['money'] > 10000000) & (df['first_name'] == 'Thibault')
```


We'll talk about this weird syntax immediately after but mind the rules above!

tell me for each index value if the number of cars is strictly higher than 0 (i.e. the person has a car)

In [57]:
has_cars = df['number_of_cars'] > 0
has_cars

id
400    False
265     True
450    False
122     True
Name: number_of_cars, dtype: bool

tell me for each index value if is_subscribed_to_newsletter is True

In [58]:
is_subscribed = df['is_subscribed_to_newsletter'] == True
is_subscribed

id
400    False
265     True
450     True
122     True
Name: is_subscribed_to_newsletter, dtype: bool

combine the two conditions in a single series

In [59]:
# combine the two conditions in a single series
# "&" NOT "and" (vector comparison, not element comparison)! Also, no need for parentheses here thanks to the use of variables.
has_cars_and_is_subscribed = has_cars & is_subscribed
has_cars_and_is_subscribed

id
400    False
265     True
450    False
122     True
dtype: bool

same thing as before in one statement

In [60]:
# == True is superfluous for column "is_subscribed_to_newsletter"
# as the label clearly indicates, it's boolean  
has_cars_and_is_subscribed = (df['number_of_cars'] > 0) & (df['is_subscribed_to_newsletter']) 
has_cars_and_is_subscribed

id
400    False
265     True
450    False
122     True
dtype: bool

get me the rows where the condition is True

In [61]:
df.loc[has_cars_and_is_subscribed, :] 

,first_name,number_of_cars,is_subscribed_to_newsletter,number_of_houses
id,,,,
265,John,2,True,1
122,Arnold,2,True,1


you can also use masks on columns but this is the kind of thing I rarely use ¯\\\_(ツ)\_/¯

In [62]:
columns_mask = df.columns.str.contains('name') # we will see this "str.contains" "thing" later
print(df.columns.tolist())
print(columns_mask)
df.loc[:, columns_mask]

['first_name', 'number_of_cars', 'is_subscribed_to_newsletter', 'number_of_houses']
[ True False False False]


,first_name
id,
400,Alex
265,John
450,Thib
122,Arnold


# 4. Basic math operations

Let's create some DataFrame. Don't mind the cryptic code to create a random (but realistic) DataFrame of newsletter data it's not very important 🙈

In [63]:
import random, uuid, math

size = 10
index = np.random.choice(['Travel', 'Hotel', 'Restaurant', 'Cinema', 'Theater', 'Cruise'], size=size)
index = pd.Index([f'{cat} Newsletter 2021 #{str(uuid.uuid4()).upper()[:8]}' for cat in index], name='newsletter')
df = pd.DataFrame(index=index)
df['recipients'] = np.random.randint(low=0, high=1500, size=size)
df['impressions'] = df['recipients'].map(lambda v: v//random.uniform(2, 4))
df['clicks'] = df['impressions'].map(lambda v: v//random.uniform(2, 4))
df['soft_bounces'] = df['recipients'].map(lambda v: v//random.uniform(20, 50))
df['hard_bounces'] = df['recipients'].map(lambda v: v//random.uniform(30, 60))
df['unsubscribes'] = df['recipients'].map(lambda v: v//random.uniform(40, 70))
df['author'] = np.random.choice(['John', 'Daniel Z.', 'Alex'], size=size)
# turns out there is a bug with argmax with the new dtype "Int64" that we'll see later
# so let's the old "int" instead 
df = df.astype({c:int for c in df.columns if c != 'author'})
df.head()

,recipients,impressions,clicks,soft_bounces,hard_bounces,unsubscribes,author
newsletter,,,,,,,
Cinema Newsletter 2021 #7552C8BF,1073,337,136,33,35,23,Daniel Z.
Cinema Newsletter 2021 #62F72997,575,182,52,19,17,8,John
Theater Newsletter 2021 #C6411A59,244,95,31,9,4,5,John
Cruise Newsletter 2021 #594B629A,151,40,10,7,3,2,Daniel Z.
Hotel Newsletter 2021 #A7B5E29A,211,85,42,8,5,4,John


## 4.1. Vector to vector operations

dividing

In [64]:
# this creates a new column "clickrate" equals to clicks/recipients
df['clickrate'] = df['clicks'] / df['recipients']
df.head()

,recipients,impressions,clicks,soft_bounces,hard_bounces,unsubscribes,author,clickrate
newsletter,,,,,,,,
Cinema Newsletter 2021 #7552C8BF,1073,337,136,33,35,23,Daniel Z.,0.126747
Cinema Newsletter 2021 #62F72997,575,182,52,19,17,8,John,0.090435
Theater Newsletter 2021 #C6411A59,244,95,31,9,4,5,John,0.127049
Cruise Newsletter 2021 #594B629A,151,40,10,7,3,2,Daniel Z.,0.066225
Hotel Newsletter 2021 #A7B5E29A,211,85,42,8,5,4,John,0.199052


addition

In [65]:
# addressees are all the people the newsletter was addressed to, no matter if delivery succeeded
df['addressees'] = df['recipients'] + df['soft_bounces'] + df['hard_bounces']
df.head()

,recipients,impressions,clicks,soft_bounces,hard_bounces,unsubscribes,author,clickrate,addressees
newsletter,,,,,,,,,
Cinema Newsletter 2021 #7552C8BF,1073,337,136,33,35,23,Daniel Z.,0.126747,1141
Cinema Newsletter 2021 #62F72997,575,182,52,19,17,8,John,0.090435,611
Theater Newsletter 2021 #C6411A59,244,95,31,9,4,5,John,0.127049,257
Cruise Newsletter 2021 #594B629A,151,40,10,7,3,2,Daniel Z.,0.066225,161
Hotel Newsletter 2021 #A7B5E29A,211,85,42,8,5,4,John,0.199052,224


you can also use multiplication, substraction, modulo, remainder etc. but I think you get the point

## 4.2. Vector and scalar (single value)

multiply clicks by 1.5 to boost our values 🐒

In [66]:
df['clicks'] * 1.5

newsletter
Cinema Newsletter 2021 #7552C8BF        204.0
Cinema Newsletter 2021 #62F72997         78.0
Theater Newsletter 2021 #C6411A59        46.5
Cruise Newsletter 2021 #594B629A         15.0
Hotel Newsletter 2021 #A7B5E29A          63.0
Restaurant Newsletter 2021 #34343E16    225.0
Cinema Newsletter 2021 #85564A80        118.5
Travel Newsletter 2021 #1C807732        145.5
Hotel Newsletter 2021 #894D578D         147.0
Cinema Newsletter 2021 #84B4F19D        124.5
Name: clicks, dtype: float64

while we're at boost the recipients AND the impressions too 🐒 (everything is multiplied by 1.5)

In [67]:
df[['recipients', 'impressions']] * 1.5

,recipients,impressions
newsletter,,
Cinema Newsletter 2021 #7552C8BF,1609.5,505.5
Cinema Newsletter 2021 #62F72997,862.5,273.0
Theater Newsletter 2021 #C6411A59,366.0,142.5
Cruise Newsletter 2021 #594B629A,226.5,60.0
Hotel Newsletter 2021 #A7B5E29A,316.5,127.5
Restaurant Newsletter 2021 #34343E16,1437.0,586.5
Cinema Newsletter 2021 #85564A80,1366.5,459.0
Travel Newsletter 2021 #1C807732,1848.0,540.0
Hotel Newsletter 2021 #894D578D,1623.0,475.5


## 4.3. Calculating sum, average, min, etc.

sum of the values of all rows for given column

In [68]:
df['clicks'].sum()

778

simple conditional sum

In [69]:
df.loc[df['author'] == 'John', 'clicks'].sum() # sum of clicks for all newsletter authored by John

275

average of clicks across all newsletter

In [70]:
df['clicks'].mean()

77.8

min clicks (so the worst value)

In [71]:
df['clicks'].min()

10

max clicks (so the best value)

In [72]:
df['clicks'].max()

150

show me the best newsletter in term of clicks

In [73]:
best_newsletter_clicks_index = df['clicks'].argmax()
print(f'index of the best newsletter by position: {best_newsletter_clicks_index}')

# I select best_newsletter_clicks_index in a list to get a DataFrame back (it looks prettier)
df.iloc[[best_newsletter_clicks_index],:]

index of the best newsletter by position: 5


,recipients,impressions,clicks,soft_bounces,hard_bounces,unsubscribes,author,clickrate,addressees
newsletter,,,,,,,,,
Restaurant Newsletter 2021 #34343E16,958,391,150,20,31,14,John,0.156576,1009


an alternative way of calculating addressees like we did before with:

```python
df['recipients'] + df['soft_bounces'] + df['hard_bounces']
```

In [74]:
df[['recipients', 'soft_bounces', 'hard_bounces']].sum(axis='columns')

newsletter
Cinema Newsletter 2021 #7552C8BF        1141
Cinema Newsletter 2021 #62F72997         611
Theater Newsletter 2021 #C6411A59        257
Cruise Newsletter 2021 #594B629A         161
Hotel Newsletter 2021 #A7B5E29A          224
Restaurant Newsletter 2021 #34343E16    1009
Cinema Newsletter 2021 #85564A80         956
Travel Newsletter 2021 #1C807732        1312
Hotel Newsletter 2021 #894D578D         1142
Cinema Newsletter 2021 #84B4F19D         718
dtype: int64

# 5. Some common DataFrame operations

## 5.1. Important note on assignment

Apart from a few exceptions such as `df.insert` (inserts a new column at a specific position) **most operations in `pandas` require assignment** or the use of the argument **`inplace`** for operations to happen. We will see examples later.

Indeed, most of the times only copies are returned. This is very interesting for:

* vizualizing transformations before executing them
* doing method chaining

## 5.2. Handling duplicates

Let's create a DataFrame with duplicates in a column that we'd want to get rid of

In [80]:
import pandas as pd

data = {'first_name':['Tibaldo', 'Tibaldo', 'John'],
        'mothertongue':['French', None, 'English']}
df = pd.DataFrame(data=data)
df

,first_name,mothertongue
0,Tibaldo,French
1,Tibaldo,None
2,John,English


### Identifying duplicates

In [88]:
display(df['first_name'])
df['first_name'].duplicated(keep=False)

0    Tibaldo
1    Tibaldo
2       John
Name: first_name, dtype: object

0     True
1     True
2    False
Name: first_name, dtype: bool

### Dropping duplicates

as I wrote with the note on the assignment before, here I am just showing what would be the result of the operation (I get a transformed copy of the DataFrame)

In [91]:
df.drop_duplicates(subset=['first_name'], keep='first') # default

,first_name,mothertongue
0,Tibaldo,French
2,John,English


see, the duplicated first name is still here

In [95]:
df

,first_name,mothertongue
0,Tibaldo,French
1,Tibaldo,None
2,John,English


with the code below it works 🐒!

Note: don't use `inplace` and `subset` together as this would raise a [SettingWithCopyWarning](https://www.dataquest.io/blog/settingwithcopywarning/) (this happens because when calling `drop_duplicates` pandas does some indexing in the background) 🙈🙈

In [96]:
df = df.drop_duplicates(subset=['first_name'], keep='first')
df

,first_name,mothertongue
0,Tibaldo,French
2,John,English


See also the [example notebook](https://github.com/ThibTrip/thib/blob/master/2021/pandas_crash_course/examples/duplicates/isolate_duplicates_excel.ipynb) where I load an Excel table into a pandas DataFrame and save duplicates in it as another Excel file.

This was a question from Irene Sanjay.

## 5.3. Handling nulls

Let's create a DataFrame with NULLs in it

In [107]:
df = pd.DataFrame({'name':['John', 'Alex', 'Daniel'],
                   'is_client':[True, pd.NA, True],
                   'number_of_cars':[2, pd.NA, pd.NA]}).convert_dtypes()
df

,name,is_client,number_of_cars
0,John,True,2
1,Alex,<NA>,<NA>
2,Daniel,True,<NA>


### Dropping nulls

In [111]:
display(df)
# drop rows where ANY of the columns mentionned in subset are null
df.dropna(how='any', # default
          subset=['is_client', 'number_of_cars'],
          axis='rows') # default

,name,is_client,number_of_cars
0,John,True,2
1,Alex,<NA>,<NA>
2,Daniel,True,<NA>


,name,is_client,number_of_cars
0,John,True,2
2,Daniel,True,<NA>


In [116]:
df.dropna(how='all', subset=['is_client', 'number_of_cars']) # drop rows where ALL of the columns mentionned in subset are null

,name,is_client,number_of_cars
0,John,True,2
2,Daniel,True,<NA>


### Identifying nulls

#### Scalars

In [117]:
pd.isna('test')

False

In [118]:
pd.isna(None)

True

In [119]:
pd.isna(pd.NaT) # Not a Time, null value for timestamps in pandas

True

In [120]:
pd.isna(np.nan) # numpy nan (not a number, type is float)

True

#### Arrays

In [122]:
display(df['is_client'])
df['is_client'].isna()

0    True
1    <NA>
2    True
Name: is_client, dtype: object

0    False
1     True
2    False
Name: is_client, dtype: bool

In [124]:
df['is_client'].isna().any()

False

In [131]:
df[['is_client', 'number_of_cars']].isna()

,is_client,number_of_cars
0,False,False
1,True,True
2,False,True


In [132]:
df[['is_client', 'number_of_cars']].isna().any(axis='columns')

0    False
1     True
2     True
dtype: bool

`pd.isna` also works on lists, tuples etc.

In [133]:
arr_isnull = pd.isna(['a', None])
display(arr_isnull)

arr_isnull.all()

array([False,  True])

False